In [12]:
from bs4 import BeautifulSoup
import requests
import tarfile
import re
import sys
import os
import pandas as pd 
import xml.etree.ElementTree as et
from collections import defaultdict

In [13]:
url = 'https://echanges.dila.gouv.fr/OPENDATA/BODACC/2020/'
r = requests.get(url, allow_redirects=True)
soup = BeautifulSoup(r.content, "lxml")
files = []

In [14]:
pages = soup.findAll('a', href=re.compile('RCS-A')) #regex to target only RCS-A files (bodacc A)
print("Total Links Found:",pages.__len__())
for l in pages:
     files.append(l.get('href'))

Total Links Found: 196


In [15]:
# itérer sur files 
test = 'RCS-A_BXA20200102.taz'
url2 = f"https://echanges.dila.gouv.fr/OPENDATA/BODACC/2020/{test}"
response = requests.get(url2, stream=True)
if response.status_code == 200:
    with open(test, 'wb') as f:
        f.write(response.raw.read())

In [16]:
def extract_tar():
    '''Extract .tar files to .xml'''
    tar = tarfile.open(test)
    tar.extractall()
    tar.close()
extract_tar()


In [17]:
def get_personnes(root):
    '''get personnes in avis '''
    for personnes in root.iter("personnes"):
        for personne in personnes.iter('personne'):
            #hommage à camacho
            try:
                choix = personne.find('personnePhysique/numeroImmatriculation/numeroIdentification')
                s_numero_identification.append(choix.text)
            except:

                try:
                    choix2 = personne.find('personneMorale/numeroImmatriculation/numeroIdentification')
                    s_numero_identification.append(choix2.text)
                    
                except:
                    s_numero_identification.append('None')          
                
                    
                                        
def get_adresse_france(root):
    '''get adresses in personnes '''
    for personnes in root.iter("personnes"):
        for personne in personnes.iter('personne'):
            try:
                cp = personne.find('adresse/france/codePostal')
                s_cp_personne.append(cp.text)
            except:
                s_cp_personne.append('None') #should append(s_cp from /get_etablissement)

            try:
                ville = personne.find('adresse/france/ville')
                s_ville_personne.append(ville.text)#should append(s_cp from /get_etablissement)

            except:
                s_ville_personne.append('None')

In [19]:
def get_actes(root):
    '''get actes in avis'''
    for acte in root.iter('acte'):
        get_creation(acte)
        get_immatriculation(acte)
        get_vente(acte)
        
def get_creation(root): 
    ''' get creation in actes'''
    
    dict_creation = {
              'dateImmatriculation':s_date_immat,
              'categorieCreation':s_categorie_creation,
              'dateCommencementActivite':s_date_commencement_activite,
              'descriptif':s_descriptif
             }
    
    for creation in root.iter('creation'):
        for key in dict_creation:
            j = creation.find(f'{key}')
            if j is None:
                dict_creation[key].append(f'No {key}')               
            else:
                dict_creation[key].append(j.text)
       
    return (s_date_immat, 
           s_categorie_creation, 
           s_date_commencement_activite,
           s_descriptif)


def get_immatriculation(root):
    ''' get immatriculation in actes'''
    
    dict_immat = {
              'dateImmatriculation': s_date_immat2,
              'dateEffet':s_date_effet,
              'categorieImmatriculation':s_categorie_immat,
              'dateCommencementActivite':s_date_commencement_activite2,
              'descriptif': s_descriptif2      
             }
    
    for immat in root.iter('immatriculation'):
        for key in dict_immat:
            j = immat.find(f'{key}')
            if j is None:
                dict_immat[key].append(f'No {key}')               
            else:
                dict_immat[key].append(j.text)
        

    return (s_date_immat2,
            s_date_effet, 
            s_categorie_immat,
            s_date_commencement_activite2, 
            s_descriptif2) 

def get_vente(root):
    ''' get vente in actes'''
    #### voir pour JAL / opposition / déclaration de créance 
    dict_vente = {
              'dateImmatriculation': s_date_immat_vente,
              'categorieVente':s_categorie_vente,
              'dateEffet':s_date_effet_vente,
              'dateCommencementActivite':s_date_commencement_activite_vente,
              'descriptif': s_descriptif_vente
               }
    
    for vente in root.iter('vente'):
        for key in dict_vente:
            j = vente.find(f'{key}')
            if j is None:
                dict_vente[key].append(f'No {key}')               
            else:
                dict_vente[key].append(j.text)
        
    return (s_date_immat_vente,
            s_categorie_vente,
            s_date_effet_vente,
            s_date_commencement_activite_vente,
            s_descriptif_vente)

In [20]:
def get_etablissement(root):
      
    '''get etablissement in avis'''
     
    dict_etablissement = {
              'activite': s_activite,
              'enseigne':s_enseigne,
              'origineFonds':s_origine_fonds,
              'qualiteEtablissement':s_qualite_etab,   
             }
    
    for etablissement in root.iter('etablissement'):
        root = etablissement
        for key in dict_etablissement:
            j = etablissement.find(f'{key}')
            if j is None:
                dict_etablissement[key].append(f'No {key}')               
            else:
                dict_etablissement[key].append(j.text)

    
        get_adresse(root)
            
            
    return (s_activite,
            s_enseigne,
            s_origine_fonds,
            s_qualite_etab,
            s_numero_voie,
            s_type_voie,s_nom_voie,
            s_code_postal,
            s_ville)
    

def get_adresse(root):
    
     '''get adress in etablissement'''
    
     dict_adresse = {
            
              'numeroVoie':s_numero_voie,
              'typeVoie':s_type_voie,
              'nomVoie':s_nom_voie,   
              'complGeographique':s_complGeographique,
              'codePostal': s_code_postal,
              'ville':s_ville
            }
    
    
     for adresse in root.iter('adresse'):
         for key in dict_adresse:
            
             j = adresse.find(f'{key}')
             if j is None:
                dict_adresse[key].append(f'No {key}')               
             else:
                dict_adresse[key].append(j.text)
        
         return (s_numero_voie,
                s_type_voie,
                s_nom_voie,
                s_complGeographique,
                s_code_postal,
                s_ville)

In [32]:
s_numero_identification = []
s_cp_personne = []
s_ville_personne = []
s_tribunal= []
s_numeroDepartement = []
s_type_annonce = [] ### OSEF 
s_origine_fonds = []
s_qualite_etab = []
s_activite = []
s_enseigne = [] 
s_numero_voie = []
s_type_voie = []
s_nom_voie = []
s_complGeographique = []
s_code_postal = []
s_ville = []
s_date_immat = []
s_categorie_creation = []
s_date_commencement_activite = []
s_descriptif = []
s_date_immat2 = []
s_categorie_immat = []
s_date_effet = []
s_date_commencement_activite2 = []
s_descriptif2 = []
s_date_immat_vente = []
s_categorie_vente = []
s_date_effet_vente = []
s_date_commencement_activite_vente = []
s_descriptif_vente = []


tree=et.parse('RCS-A_BXA20200102.xml')
root=tree.getroot()

for i in root.iter("avis"):
    root1 = i 
    
    tribunal = root1.find('tribunal')
    if tribunal is None:
        s_tribunal.append('None')
    else: 
        s_tribunal.append(tribunal.text)
            
    n_dep = root1.find('numeroDepartement')
    if n_dep is None:
        s_numeroDepartement.append('None')
    else:
        s_numeroDepartement.append(n_dep.text)

    get_personnes(root1)
    get_adresse_france(root1)
    get_etablissement(root1)
    get_actes(root1)  
    
    

In [33]:
df5 = pd.DataFrame({
         'numero d identification': len(s_numero_identification),
         's_cp_personne' : len(s_cp_personne), 
         's_ville_personne': len(s_ville_personne), 
         'tribunal':len(s_tribunal),
         'numero dpt': len(s_numeroDepartement),
         'origine des fonds':len(s_origine_fonds),
         'qualite etablissement': len(s_qualite_etab), 
         'activite enseigne':len(s_activite),
         'enseigne':len(s_enseigne),
         'n°':len(s_numero_voie),
         'type voie':len(s_type_voie),
         'nom de voie':len(s_nom_voie),
         'complément': len(s_complGeographique),
         'cp':len(s_code_postal),
         'ville':len(s_ville),
         'date immat création': len(s_date_immat),
         'categorie creation':len(s_categorie_creation),
         'date commencement activite':len(s_date_commencement_activite),
         'descriptif':len(s_descriptif),
         'date immat2':len(s_date_immat2),
         'date effet':len(s_date_effet),
         'cat immat':len(s_categorie_immat),
         'date commencement activi':len(s_date_commencement_activite2),
         'descriptif2':len(s_descriptif2),
         'date immat vente':len(s_date_immat_vente),
         'cat vente':len(s_categorie_vente),
         'date effet vente': len(s_date_effet_vente),
         'date comm activite vente':len(s_date_commencement_activite_vente),
         'descirptif vente':len(s_descriptif_vente)
},index=["nb_lignes"]).T


df5

,nb_lignes
numero d identification,1246
s_cp_personne,1246
s_ville_personne,1246
tribunal,1246
numero dpt,1246
origine des fonds,1145
qualite etablissement,1145
activite enseigne,1145
enseigne,1145
n°,1111


In [34]:
liste_test = [
s_numero_identification,
s_cp_personne,
s_ville_personne,
s_tribunal,
s_numeroDepartement,
s_origine_fonds, 
s_qualite_etab, 
s_activite, 
s_enseigne, 
s_numero_voie, 
s_type_voie,
s_nom_voie,
s_complGeographique,
s_code_postal,
s_ville,
s_date_immat,
s_categorie_creation,
s_date_commencement_activite,
s_descriptif,
s_date_immat2,
s_categorie_immat,
s_date_effet,
s_date_commencement_activite2,
s_descriptif2,
s_date_immat_vente,
s_categorie_vente,
s_date_effet_vente,
s_date_commencement_activite_vente,
s_descriptif_vente
]    
for l in range(20):
    print('-------------------------')
    for i in liste_test:
        print(i[l])


-------------------------
850 828 005
01630
Saint-Genis-Pouilly
GREFFE DU TRIBUNAL DE COMMERCE DE BOURG-EN-BRESSE
01
Fonds artisanal acquis par apport au montant évalué à 229500.00 euros
établissement principal
boucherie charcuterie traiteur
No enseigne
6
Rue
de Genève
No complGeographique
01630
Saint-Genis-Pouilly
2020-05-19
Immatriculation d'une personne physique suite à création d'un établissement principal
2020-05-18
No descriptif
2020-05-22
Immatriculation d'une personne morale suite à transfert de son siège social
No dateEffet
2020-01-20
Immatriculation suite à transfert de son siège social hors ressort. Modification survenue sur l'administration
No dateImmatriculation
Achat d'un fonds par une personne morale (insertion provisoire)
No dateEffet
2019-07-01
Acte en date du 10/01/2020 enregistré au SPFE DE BOURG EN BRESSE le 26/02/2020 sous le numéro 2020 0010119 référence 0104P01 Adresse de l'ancien propriétaire: 6 Rue de Genève 01630 Saint-Genis-Pouilly
-------------------------
8

In [35]:
df1 = pd.DataFrame({'Tribunal': s_tribunal,
                    'departement':s_numeroDepartement})
df1

,Tribunal,departement
0,GREFFE DU TRIBUNAL DE COMMERCE DE BOURG-EN-BRESSE,01
1,GREFFE DU TRIBUNAL DE COMMERCE DE ST-QUENTIN,02
2,GREFFE DU TRIBUNAL DE COMMERCE DE ST-QUENTIN,02
3,GREFFE DU TRIBUNAL DE COMMERCE DE ST-QUENTIN,02
4,GREFFE DU TRIBUNAL DE COMMERCE DE ST-QUENTIN,02
...,...,...
1241,GREFFE DU TRIBUNAL DE COMMERCE DE CAYENNE,973
1242,GREFFE DU TRIBUNAL MIXTE DE COMMERCE DE SAINT-...,974
1243,GREFFE DU TRIBUNAL MIXTE DE COMMERCE DE SAINT-...,974
1244,GREFFE DU TRIBUNAL MIXTE DE COMMERCE DE SAINT-...,974


In [36]:
df2 = pd.DataFrame({'enseigne': s_enseigne,
                    'activité':s_activite,
                    'origine des fonds':s_origine_fonds,
                    'qualite_etablissement':s_qualite_etab})
#df2

In [37]:
df3 = pd.DataFrame({'n°':s_numero_voie,
                    'type de voie':s_type_voie, 
                    'nom voie': s_nom_voie,
                    'cp': s_code_postal, 
                    'ville': s_ville})

In [38]:
pd.set_option('display.max_columns', None)
df3.head(50)

,n°,type de voie,nom voie,cp,ville
0,6,Rue,de Genève,01630,Saint-Genis-Pouilly
1,47,rue,de l'Epine,02680,Dallon
2,11,rue,René Coty,02300,Chauny
3,7,rue,de la Barriere,02000,Aulnois-sous-Laon
4,No numeroVoie,chemin,des Chennevieres,02210,Coincy
5,12,rue,de la Mare aux Pigeons,02210,Coincy
6,11 bis,rue,du Paradis,02400,Château-Thierry
7,No numeroVoie,No typeVoie,No nomVoie,04310,Peyruis
8,No numeroVoie,rue,du Tribunal,04100,Manosque
9,7,rue,des Petites Aires,04800,Saint-Martin-de-Bromes


In [39]:
df4 = pd.DataFrame({'dateImmatriculation':s_date_immat,
            'categorieCreation':s_categorie_creation,
            'dateCommencementActivite':s_date_commencement_activite,
            'descriptif':s_descriptif
            })
df4

,dateImmatriculation,categorieCreation,dateCommencementActivite,descriptif
0,2020-05-19,Immatriculation d'une personne physique suite ...,2020-05-18,No descriptif
1,2020-05-19,"Immatriculation d'une personne morale (B, D) s...",No dateCommencementActivite,Cette société n'exerce aucune activité
2,2020-05-19,"Immatriculation d'une personne morale (B, C, D...",2020-04-02,No descriptif
3,2020-05-19,"Immatriculation d'une personne morale (B, C, D...",2020-02-14,No descriptif
4,2020-05-19,Immatriculation d'une personne physique suite ...,2020-05-11,No descriptif
...,...,...,...,...
1020,2020-05-18,"Immatriculation d'une personne morale (B, C, D...",2020-04-22,No descriptif
1021,2020-05-18,"Immatriculation d'une personne morale (B, C, D...",2020-04-22,No descriptif
1022,2020-05-19,"Immatriculation d'une personne morale (B, C, D...",2020-06-02,No descriptif
1023,2020-05-19,"Immatriculation d'une personne morale (B, C, D...",2020-03-05,No descriptif


In [41]:
df_final = pd.DataFrame({'n° RCS': s_numero_identification,
             'code postal': s_cp_personne,
             'ville':s_ville_personne, 
             #'date de création':s_date_creation
             
            })

<bound method DataFrame.nunique of            n° RCS code postal                ville
0     850 828 005       01630  Saint-Genis-Pouilly
1     883 520 611        None                 None
2     883 499 675       02260           La Capelle
3     883 513 517       02300               Chauny
4     883 526 733       02000    Aulnois-sous-Laon
...           ...         ...                  ...
1241  883 274 870       97300              Cayenne
1242  883 502 429       97480         Saint-Joseph
1243  883 521 973       97427         L'Étang-Salé
1244  883 502 585       97450          Saint-Louis
1245  847 948 569        None                 None

[1246 rows x 3 columns]>